# Inference Sample for EquiDock

SPDX-FileCopyrightText: Copyright (c) <year> NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: LicenseRef-NvidiaProprietary

NVIDIA CORPORATION, its affiliates and licensors retain all intellectual property and proprietary rights in and to this material, related documentation and any modifications thereto. Any use, reproduction, disclosure or distribution of this material and related documentation without an express license agreement from NVIDIA CORPORATION or its affiliates is strictly prohibited.


## Large Workload

If you have large workload, you can use `infer.py` located at `examples/protein/equidock/infer.py`. 

The inputs to `infer.py` is two zip file each containing PDB files, zip file should be named `ligands.zip` and `receptors.zip`. The PDB files for each complex are named `{PP-Common-name}_l_b.pdb` inside ligands.zip and `{PP-Common-name}_r_b.pdb` inside receptros.zip example below,

ligand_file = "a9_1a95.pdb1_3.dill_l_b.pdb"
receptor_file = "a9_1a95.pdb1_3.dill_r_b.pdb"

The `infer.py` then returns displaced and rotated ligand protein as PDB file. To use the model with `infer.py` with a new dataset (zip files), one should modify configurations yaml file located under `/examples/protein/equidock/conf/infer.yaml` file. Specifically, following lines.

```
data:
  data_name: db5 # db5, dips for inference
  …
  data_dir: ${oc.env:BIONEMO_HOME}/tests/equidock_test_data/test_sets_pdb/${data.data_name}_test_random_transformed/random_transformed # folder for new dataset, storing zip files, see above for the naming convention.
```
The results are written in the exp_dir specified in the same yaml file.

:::{note} The Equidock model assumes that the receptor protein in the PP-Complex is fixed in space and does not move; it only return the ligand protein coordinates.


#### Import

Components for inferencing are part of the BioNeMo EquiDock source code. This notebook demonstrates the use of these components.

In [ ]:
import numpy as np
from pytriton.client import ModelClient
from zipfile import ZipFile
import os
import tempfile

## Start inference server
`python examples/protein/equidock/nbs/infer_server.py --model={model_name} &`

`model_name` can be set to one of the two provided models:
1. `dips`: Model trained on Databse of Interacting Protein Structures (DIPS)
2. `db5`: Model trained on Docking Benchmark 5.5 (DB5.5)

## Note on steric clashes

Some clashes are possible in EquiDock model. EquiDock paper suggests to use SGD optimizer on intersection loss to find optimal translation and rotation. To remove clashes, set flag in postprocessing section of `infer.yaml` or see below.

```
cfg.postprocess.remove_clashes = True
cfg.postprocess.min_loss = 0.5
cfg.postprocess.max_iteration = 2000
cfg.postprocess.lr = 0.001
cfg.postprocess.fast_optimizer = True
```

## Inputs

### Set input ligand and receptor PDB files

In [ ]:
## In this notebook example, we will use ligand and receptors files in the test directory
test_folder_location = os.path.join(os.path.dirname(os.getcwd()),  "../../../") # test directory
zip_file_path = "tests/equidock_test_data/test_sets_pdb/dips_test_random_transformed/random_transformed" 

In [ ]:
ligands_zip = os.path.join(test_folder_location, zip_file_path, "ligands.zip")
receptors_zip = os.path.join(test_folder_location, zip_file_path, "receptors.zip")

# Temporary output folder
tmp_output_folder = tempfile.TemporaryDirectory()

# Select ligand filename for extraction
ligand_file = "a9_1a95.pdb1_3.dill_l_b.pdb"
with ZipFile(ligands_zip, "r") as zipper:
    zipper.extract(ligand_file, tmp_output_folder.name)

print(f"Ligand extracted to {os.path.join(tmp_output_folder.name, ligand_file)}")
# Receptor filename for extraction
receptor_file = "a9_1a95.pdb1_3.dill_r_b.pdb"
with ZipFile(receptors_zip, "r") as zipper:
    zipper.extract(receptor_file, tmp_output_folder.name)
print(f"Receptor extracted to {os.path.join(tmp_output_folder.name, receptor_file)}")

## Ouputs
### Set output complex PDB file

In [ ]:
out_file = "a9_1a95.pdb1_3.equidock_complex.pdb"
print(f"Output will be written to {os.path.join(tmp_output_folder.name, out_file)}")

### Format input/outputs for `pytriton` client

In [ ]:
ligand_filename = np.array([[os.path.join(tmp_output_folder.name, ligand_file)]])
ligand_filename = np.char.encode(ligand_filename, "utf-8")

receptor_filename = np.array([[os.path.join(tmp_output_folder.name, receptor_file)]])
receptor_filename = np.char.encode(receptor_filename, "utf-8")

out_filename = np.array([[os.path.join(tmp_output_folder.name, out_file)]])
out_filename = np.char.encode(out_filename, "utf-8")    

###  Use ModelClient to run inference on the server

EquiDock predicted complex written to `out_filename`

In [ ]:
with ModelClient("localhost", "bionemo_model") as client:
    _ = client.infer_batch(ligand_filename, receptor_filename, out_filename)

In [ ]:
# Install py3Dmol for visualization
try:
    import py3Dmol
except:
    !pip install py3Dmol
    import py3Dmol

In [ ]:
# Load ligand, receptor, and predicted complex files for visualization
with open(os.path.join(tmp_output_folder.name, receptor_file)) as ifile:
    receptor = "".join([x for x in ifile])
    
with open(os.path.join(tmp_output_folder.name, ligand_file)) as ifile:
    ligand = "".join([x for x in ifile])

with open(os.path.join(tmp_output_folder.name, out_file)) as ifile:
    equidock_complex = "".join([x for x in ifile])

In [ ]:
print("Receptor")
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(receptor)
view.setStyle({'model': -1}, {"cartoon": {'color': 'purple'}})
view.zoomTo()
view.show()

In [ ]:
print("Ligand")
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(ligand)
view.setStyle({'model': -1}, {"cartoon": {'color': 'green'}})
view.zoomTo()
view.show()

In [ ]:
print("EquiDock complex")
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(receptor)
view.setStyle({'model': 0}, {"cartoon": {'color': 'purple'}})
view.addModelsAsFrames(equidock_complex)
view.setStyle({'model': 1}, {"cartoon": {'color': 'green'}})
view.zoomTo()
view.show()

In [ ]:
tmp_output_folder.cleanup()